# LAB: Comparación de modelos

En este lab vamos a comparar el rendimiento de todos los modelos que hemos aprendido hasta ahora, utilizando el dataset de evaluación de autos.

## 1. Preparación de los datos

El [dataset de evaluación de autos](https://archive.ics.uci.edu/ml/machine-learning-databases/car/) está en la carpeta datasets. Ya debería estar familiarizado con el mismo.

1. Leer los datos en un dataframe de pandas
- Codificar adecuadamente los features categóricos: definir un mapeo que preserve el orden (asignando números más pequeños a palabras que indiquen cantidades más pequeñas)
- Separar los features de la variable objetivo en X e y

In [1]:
import pandas as pd
df = pd.read_csv('../Data/car.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying           1728 non-null object
maint            1728 non-null object
doors            1728 non-null object
persons          1728 non-null object
lug_boot         1728 non-null object
safety           1728 non-null object
acceptability    1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


In [3]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,acceptability
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,med,med,4,more,big,med,unacc
freq,432,432,432,576,576,576,1210


In [4]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [5]:
df.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety',
       'acceptability'],
      dtype='object')

In [6]:
for col in df.columns:
    print(col, df[col].unique())

buying ['vhigh' 'high' 'med' 'low']
maint ['vhigh' 'high' 'med' 'low']
doors ['2' '3' '4' '5more']
persons ['2' '4' 'more']
lug_boot ['small' 'med' 'big']
safety ['low' 'med' 'high']
acceptability ['unacc' 'acc' 'vgood' 'good']


In [7]:
codingMap = {
    'low': 0,
    'med': 1,
    'high': 2,
    'vhigh': 3,
    '2': 2,
    '3': 3,
    '4': 4,
    '5more': 5,
    'more': 6,
    'small': 0,
    'big': 2,
    'unacc': 0,
    'acc': 1,
    'good': 2,
    'vgood': 3
}

df2 = df.applymap(lambda v: codingMap[v])

In [8]:
X = df2[['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']]
y = df2['acceptability']

In [9]:
df2.head()

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,3,3,2,2,0,0,0
1,3,3,2,2,0,1,0
2,3,3,2,2,0,2,0
3,3,3,2,2,1,0,0
4,3,3,2,2,1,1,0


## 2. Preparación útil

Dado que vamos a comparar varios modelos, resulta conveniente  escribir funciones auxiliares.

1. Separar X e y entre entrenamiento / prueba, usando el 30% como set de prueba, y asignando `random_state=42`
    - Asegurar de que los datos son mezclados y estratificados
2. Definir una función llamada `evaluate_model`, que entrene el modelo con el set de entrenamiento y lo evalué con el set de prueba. Y que calcule:
    - accuracy score
    - confusion matrix
    - classification report
3. Inicializar un diccionario global para almacenar los distintos modelos, para su posterior uso.


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

modelsDict = {}

def evaluate_model(label, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    
    modelsDict[label] = {
        model: model,
        accuracy: accuracy
    }
    
    print(accuracy)
    print(confusion)

## 3.a KNN

Comenzar con `KNeighborsClassifier`.

1. Inicializar un modelo KNN
- Evaluar su rendimiento con la función definida previamente
- Buscar el valor óptimo de K mediante Grid Search
    - Tener cuidado en cómo realizar la validación cruzada en Grid Search

In [18]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()

evaluate_model('KNN', neigh)

0.938342967245
[[357   6   0   0]
 [  8 106   1   0]
 [  1  11   9   0]
 [  0   4   1  15]]


In [13]:
KNeighborsClassifier?

In [15]:
import numpy as np
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(neigh, {
    'n_neighbors': [n for n in range(1, 100)],
}, verbose=1)

gs.fit(X, y)

Fitting 3 folds for each of 99 candidates, totalling 297 fits


[Parallel(n_jobs=1)]: Done 297 out of 297 | elapsed:   10.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [16]:
gs.best_params_

{'n_neighbors': 36}

In [27]:
neigh = KNeighborsClassifier(n_neighbors=5)

evaluate_model('KNN', neigh)

0.938342967245
[[357   6   0   0]
 [  8 106   1   0]
 [  1  11   9   0]
 [  0   4   1  15]]


## 3.b Bagging + KNN

Habiendo encontrado el K óptimo, vamos a meter `KNeighborsClassifier` en un BaggingClassifier y ver si la puntuación mejora.

1. Usar el modelo KNN en un Bagging Classifier
- Evaluar el rendimiento
- Hacer un Grid Search sólo en los parámetros del Bagging Classifier

In [29]:
from sklearn.ensemble import BaggingClassifier
bag = BaggingClassifier(base_estimator=KNeighborsClassifier())

In [30]:
evaluate_model('Bagging of KNN', bag)

0.922928709056
[[355   8   0   0]
 [ 11 102   2   0]
 [  0  11   9   1]
 [  0   5   2  13]]


## 4. Regresión Logística

Ahora veamos si la regresión logística funciona mejor

1. Inicializar una Regresión Logística y evaluarla
- Buscar los parámetros óptimos con Grid Search
- Ver si Bagging mejora el resultado

## 5. Árboles de decisión

Comprobemos como rinden los Árboles de Decisión

1. Inicializar un árbol de decisión y evaluarlo
- Buscar los parámetros óptimos con Grid Search
- Ver si Bagging mejora el resultado

## 6. Support Vector Machines

Analicemos como funciona SVM

1. Inicializar un SVM y evaluarlo
- Buscar los parámetros óptimos con Grid Search
- Ver si Bagging mejora el resultado

## 7. Random Forest & Extra Trees

Ahora veamos por último como funcionan Random Forest y Extra Trees

1. Inicializar RF y ET y evaluarlos
- Buscar los parámetros óptimos con Grid Search

## 8. Comparación de modelos

Vamos a comparar los resultados de los diferentes modelos.

1. Haga un gráfico de barras de los resultados de los mejores modelos. ¿Cuál es el ganador?
- Volver a evaluar todos los modelos usando una validación cruzada estratificada de 3-fold
- Hacer un gráfico de barras, mostrando el valor medio y desvío del resultado de la validación cruzada. ¿Es el mismo ganador?


## Extra

Hemos codificado los datos utilizando un mapa que preservaba el orden.
¿Cómo cambiarían los resultados si codificamos los datos categóricos utilizando `pd.get_dummies` o `OneHotEncoder` para codificarlos como variables binarias?

1. Repetir el análisis para este escenario. ¿Es mejor?
- Experimentar con otros modelos u otros parámetros, ¿puedes obtener el mejor resultado de la clase?